<a href="https://colab.research.google.com/github/Jaihyun-Byun/NLP-Project/blob/main/Day5_seesion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 Comprehensive Agentic NLP Tutorial: From LLMs to Agents

안녕하세요! 이 튜토리얼은 **단순한 챗봇(LLM)을 넘어, 스스로 생각하고 도구를 사용하며 복잡한 문제를 해결하는 'AI 에이전트'**를 만드는 과정을 다룹니다.

## 📚 무엇을 배우나요?
이 노트북 하나로 다음의 핵심 기술들을 마스터할 수 있습니다.

1.  **Code Generation**: LLM이 코드를 짜고, *스스로 에러를 고치는* 방법
2.  **Advanced RAG**: 단순 검색을 넘어, *질문을 여러 개로 쪼개어* 더 정확하게 검색하는 법
3.  **Toos & MCP**: AI에게 계산기나 검색 엔진 같은 *'손과 발'*을 달아주는 표준 규격(MCP)
4.  **Multi-Agent Systems**: 혼자서 못하는 일을 *'팀(Team)'*을 꾸려 해결하는 다양한 패턴 (ReAct, Reflexion 등)
5.  **Advanced Features**: AI에게 *기억력(Memory)*을 주고, *사람의 결재(Human-in-the-loop)*를 받고, *성능을 채점(Evaluation)*하는 법

---
## ⚠️ 준비물
- **Google Colab (T4 GPU 이상 권장)** 또는 로컬 환경
- **Ollama**: 로컬 LLM 서버가 켜져 있어야 합니다. (실습 환경에 따라 설정 필요)

## 1. Environment Setup (환경 설정)

요리를 하려면 도구랑 재료가 필요하죠? AI 에이전트를 만들기 위한 라이브러리들을 설치하고, 우리의 두뇌가 될 **Ollama(LLM)**를 연결합니다.

In [ ]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > ollama.log &

In [ ]:
# LangChain 및 MCP 관련 패키지 설치
!pip install --quiet -U openai ollama \
  langchain langchain_community langchain_core langchain_openai langchainhub langchain_elasticsearch langchain_ollama \
  transformers datasets python-dotenv tenacity google-search-results unstructured arxiv pymupdf tiktoken \
  streamlit streamlit-folium wikipedia jq ragas dspy \
  mcp langchain-mcp-adapters langgraph langchain_tavily langchain_experimental \
  langchain-chroma langchain-community google-search-results pymysql

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: redirecting stderr to stdout


In [ ]:
!nvidia-smi

Wed Feb  4 11:05:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 기본 설정
import os
from langchain_ollama import ChatOllama

# Ollama가 실행 중인 서버 URL
BASE_URL = "http://localhost:11434"

# 💡 모델 선택
# - llama3.1: 일반적인 대화와 추론을 잘함 (문과생 느낌)
# - qwen2.5-coder: 코딩을 기가 막히게 잘함 (이과생 느낌)
LLM_NAME = "llama3.1:8b"
CODE_LLM_NAME = "qwen2.5-coder:7b"
EMBEDDING_NAME = "nomic-embed-text"


# 모델 다운로드 (최초 1회 실행 필요)
!ollama pull $LLM_NAME
!ollama pull $CODE_LLM_NAME
!ollama pull $EMBEDDING_NAME

# LangChain LLM 인스턴스 생성
llm = ChatOllama(
    model=LLM_NAME,
    temperature=0,
    base_url=BASE_URL
)

code_llm = ChatOllama(
    model=CODE_LLM_NAME,
    temperature=0.1,
    base_url=BASE_URL
)

embedding = OllamaEmbeddings(model="nomic-embed-text", base_url=BASE_URL)

print(f"준비 완료! 대화용 모델: {LLM_NAME}, 코딩용 모델: {CODE_LLM_NAME}, 임베딩 모델: {EMBEDDING_NAME}")




준비 완료! 대화용 모델: llama3.1:8b, 코딩용 모델: qwen2.5-coder:7b, 임베딩 모델: nomic-embed-text


In [ ]:
# Helper function: 다이어그램(Mermaid)을 노트북에서 바로 보여주는 함수
import base64
from IPython.display import Image, display

def mm(graph):
    graphbytes = graph.encode("utf-8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

## 2. Code Generation (코드 생성 및 자가 수정)

### 🐣 개념 설명: "한 번에 완벽한 코드는 없다"
코딩 과제를 할 때, 코드를 한 번 짜고 바로 제출하나요? 아니죠.
**실행해보고(Run) -> 에러가 나면(Error) -> 고칩니다(Debug)**.

AI도 마찬가지입니다. 단순히 "코드 짜줘"라고 하는 것보다, **"코드 짜고 실행해봐. 에러 나면 고쳐서 다시 실행해봐"**라고 시키는 것이 훨씬 결과가 좋습니다. 이것이 바로 **Self-Correction(자가 수정) Agent**의 기초입니다.

**📚 관련 연구:**
> **Self-Refine: Iterative Refinement with Self-Feedback** (Madaan et al., 2023)  
> [[Paper Link]](https://arxiv.org/abs/2303.17651)
> *"LLM이 쓴 글이나 코드를 스스로 피드백을 주고 고치게(Refine) 했더니 성능이 훨씬 좋아지더라."*

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 기본적인 코드 생성 프롬프트
code_gen_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert Python programmer. Return ONLY the code inside markdown code blocks. No explanation."),
    ("user", "{task}")
])

code_chain = code_gen_prompt | code_llm | StrOutputParser()

task = "Write a Python function to calculate the N-th Fibonacci number efficiently."
generated_code = code_chain.invoke({"task": task})
print("### 1차 생성된 코드 ###")
print(generated_code)

### 1차 생성된 코드 ###
```python
def fibonacci(n):
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b
```


### 🛠️ 실습: 자가 수정(Self-Correction) 루프 구현
아래 코드는 AI가 짠 코드를 실제로 실행해보고(`PythonREPL`), 에러가 발생하면 에러 메시지를 다시 AI에게 보여주며 "이거 에러 났어. 고쳐줘"라고 요청하는 과정을 자동화한 것입니다.

In [ ]:
from langchain_experimental.utilities import PythonREPL
import re

repl = PythonREPL()

def extract_code(text):
    # 마크다운 코드 블록에서 순수 파이썬 코드만 발라내는 함수
    pattern = r"```python\n(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    return text.replace("```", "")

def code_repair_loop(task, max_retries=3):
    current_code = code_chain.invoke({"task": task})

    for i in range(max_retries):
        clean_code = extract_code(current_code)
        print(f"\n--- 시도 {i+1}회차: 실행 중... ---/")
        try:
            result = repl.run(clean_code)

            # 에러가 없는지 확인
            if "Error" not in result and "Traceback" not in result:
                print("🎉 성공! 실행 결과:", result)
                return clean_code
            else:
                print("💥 실행 실패 (에러 발생):", result)
                # 💡 핵심: 에러 결과를 다시 프롬프트에 넣어서 수정을 요청함
                fix_prompt = f"The following python code failed with this error:\n{result}\n\nCode:\n{clean_code}\n\nFix the code and return only the full fixed code."
                current_code = code_chain.invoke({"task": fix_prompt})
        except Exception as e:
            print("System Error:", e)
            break
    return None

complex_task = "Write a python script that creates a list of 5 random numbers and prints their sorted values. Use the 'random' module."
final_code = code_repair_loop(complex_task)


--- 시도 1회차: 실행 중... ---/
🎉 성공! 실행 결과: Random Numbers: [84, 52, 7, 65, 68]
Sorted Numbers: [7, 52, 65, 68, 84]



## 3. Advanced RAG (검색 증강 생성 심화)

### 🐣 개념 설명: "오픈북 시험의 달인"
**RAG**는 LLM에게 '교과서(문서)'를 쥐어주는 것입니다.
하지만 질문이 애매하면 교과서 어디를 봐야 할지 모릅니다.

- **Basic RAG**: "이거 찾아줘" -> (키워드 매칭) -> "여기 있습니다."
- **Advanced RAG (MultiQuery)**: "이거랑 비슷한 거, 관련된 거, 반대되는 거 다 찾아봐." -> 질문을 3~4개로 뻥튀기해서 구석구석 다 찾아봄.

질문을 다양하게 변형해서 검색하면, 놓치는 정보 없이 훨씬 정확한 답을 얻을 수 있습니다.

**📚 관련 연구:**
> **Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks** (Lewis et al., 2020)  
> [[Paper Link]](https://arxiv.org/abs/2005.11401)
> *RAG 개념을 처음 정립한 기념비적인 논문입니다.*

<img src="https://lilianweng.github.io/posts/2023-06-23-agent/agent-overview.png" width="600" alt="Agent Overview">  
*(이미지 출처: [Lilian Weng Blog](https://lilianweng.github.io/posts/2023-06-23-agent/) - 자율 에이전트 시스템 개요)*

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 1. 데이터 로드 (Lilian Weng의 블로그 포스트)
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"
]
loader = WebBaseLoader(urls)
data = loader.load()

# 2. 문서 쪼개기 (너무 길면 못 읽으니까)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)


문서 색인 완료! 이제 검색할 수 있습니다.


In [ ]:
splits[:2]

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resu

In [ ]:
from langchain_core.documents import Document

retriever = TFIDFRetriever.from_documents(
    splits
)

In [ ]:
question = "What are the main components of an LLM-powered agent?"
docs = retriever.invoke(question)
print(f"\nRetrieved {len(docs)} documents using TF-IDF.")


Retrieved 4 documents using TF-IDF.


In [ ]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resu

In [ ]:
# 4. 벡터 저장소 생성 (도서관 책장)
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from langchain_community.retrievers import TFIDFRetriever
from langchain_core.documents import Document
import logging

logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Test
question = "What are the main components of an LLM-powered agent?"
print(f"Original Question: {question}\n")

docs = vectorstore.similarity_search(question)
print(f"\nRetrieved {len(docs)} documents using MultiQuery.")

Original Question: What are the main components of an LLM-powered agent?


Retrieved 4 documents using MultiQuery.


In [ ]:
docs

[Document(id='0969578a-1ff5-4d81-a8d6-ca9955e7dd3a', metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# RAG 체인 조립
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke(question)
print("### AI의 답변 ###")
print(response)

### AI의 답변 ###
The main components of an LLM-powered agent are:

1. Planning
	* Subgoal and decomposition: breaking down large tasks into smaller, manageable subgoals.
	* Reflection and refinement: self-criticism, self-reflection, learning from mistakes, and refining actions for future steps.
2. Memory
	* Short-term memory: utilizing in-context learning to learn new information.
	* Long-term memory: retaining and recalling (infinite) information over extended periods using an external vector store and fast retrieval.
3. Tool use: the agent learns to call external APIs for extra information that is missing from the model weights, including current information, code execution capability, access to proprietary information sources, etc.


## 4. Tools & MCP (AI에게 손과 발 달아주기)

### 🐣 개념 설명: "뇌만 있는 천재" vs "컴퓨터를 쓰는 평재"
LLM은 뇌만 둥둥 떠있는 상태와 같습니다. 계산도 잘 못하고(4자리수 곱셈 틀림), 오늘 날씨도 모릅니다.
AI에게 **계산기(Tool)**나 **검색엔진(Tool)**을 쥐어주면 훨씬 똑똑해집니다.

그런데, 계산기 연결하는 코드 짜고, 검색엔진 연결하는 코드 짜고... 너무 귀찮죠?
**MCP (Model Context Protocol)**는 **'AI 도구용 USB 표준'**입니다.
마우스나 키보드를 USB에 꽂으면 바로 동작하듯이, MCP 표준을 따르는 도구는 어떤 AI에도 바로 연결할 수 있습니다.

**📚 관련 연구:**
> **Toolformer: Language Models Can Teach Themselves to Use Tools** (Schick et al., 2023)  
> [[Paper Link]](https://arxiv.org/abs/2302.04761)
> *"모델이 스스로 API를 호출하는 법을 배울 수 있다는 것을 증명한 논문"*

In [ ]:
from langchain_core.tools import tool

### 4.1 기본적인 커스텀 도구 정의 (@tool 데코레이터 사용)

@tool
def calculate_length(text: str) -> int:
    """Calculates the length of the given text string."""
    return len(text)

@tool
def reverse_string(text: str) -> str:
    """Reverses the given text string."""
    return text[::-1]

tools = [calculate_length, reverse_string]

# LLM에게 도구 쥐어주기 (bind_tools)
# 이제 LLM은 필요할 때 스스로 이 파이썬 함수를 호출하겠다고 말할 수 있습니다.
llm_with_tools = llm.bind_tools(tools)

query = "What is the length of 'Antigravity'? And also reverse it."
ai_msg = llm_with_tools.invoke(query)

# AI가 직접 답하는 게 아니라, "이 도구 좀 써주세요"라고 요청(Calls)합니다.
print(f"Tool Calls: {ai_msg.tool_calls}")

Tool Calls: [{'name': 'calculate_length', 'args': {'text': 'Antigravity'}, 'id': 'c0f3a534-7868-46ad-9caf-ff7f15bf77c7', 'type': 'tool_call'}, {'name': 'reverse_string', 'args': {'text': 'Antigravity'}, 'id': 'e5af7f63-cc51-4404-886e-9e1c98fad053', 'type': 'tool_call'}]


### 4.2 Model Context Protocol (MCP) 실습
이제 로컬 파이썬 스크립트로 **'가짜 서버'**를 만들고, LangChain 클라이언트로 접속해봅니다.
이 구조는 내 컴퓨터의 파일 시스템을 AI에게 열어주거나, 회사 내부 DB를 AI에게 연결할 때 표준적으로 사용됩니다.

In [ ]:
%%writefile simple_mcp_server.py
# 1. MCP 서버 코드 작성 (별도 파일로 저장)
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("SimpleMath")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

if __name__ == "__main__":
    mcp.run()

Overwriting simple_mcp_server.py


In [ ]:
# 2. MCP Client로 위 서버에 접속
from langchain_mcp_adapters.client import MultiServerMCPClient
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# 서버 실행 파라미터
server_params = StdioServerParameters(
    command="python",
    args=["simple_mcp_server.py"],
)

async def use_mcp_tools():
    # python 프로세스로 서버를 띄우고 통신합니다.
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # 서버가 무슨 도구를 가지고 있는지 물어봅니다.
            tools = await session.list_tools()
            print(f"Connected to MCP Server. Available Tools: {[t.name for t in tools.tools]}")

            # 도구를 사용해봅니다.
            result = await session.call_tool("add", arguments={"a": 10, "b": 20})
            print(f"Result of 'add(10, 20)': {result.content}")

# await use_mcp_tools() # 실행하려면 주석 해제 (Colab/Jupyter는 이미 비동기 환경이라 await 필요)

## 5. Multi-Agent Systems & Skills (AI 협업)

**"빨리 가려면 혼자 가고, 멀리 가려면 함께 가라"**는 말이 있죠.
복잡한 문제는 혼자 풀기 어렵습니다. 역할을 나누고 협업하는 다양한 **'에이전트 디자인 패턴'**을 배웁니다.

### 5.1 ReAct (Reason + Act)

#### 🐣 비유: "생각하고 말해!"
그냥 생각나는 대로 말하는(답변하는) AI는 실수를 많이 합니다.
**ReAct**는 답변하기 전에 속으로 **Plan(계획)**을 짜고, **Act(실행)** 해보고, **Observe(관찰)**한 뒤에 최종 답변을 합니다.

1. **Thought**: "사용자가 20 + 20을 물어봤네. 계산기 도구를 써야겠다."
2. **Action**: `Calculator(20, 20)` 호출
3. **Observation**: 결과 `40` 확인
4. **Answer**: "정답은 40입니다."

**📚 관련 연구:**
> **ReAct: Synergizing Reasoning and Acting in Language Models** (Yao et al., 2022)
> [[Paper Link]](https://arxiv.org/abs/2210.03629)

In [ ]:
mm("""
graph TD
    Start --> Thought
    Thought -->|어? 도구가 필요하네| Action
    Action --> Tool((도구 사용))
    Tool --> Observation
    Observation --> Thought
    Thought -->|이제 답을 알겠어| Finish
""")

In [ ]:
from langgraph.prebuilt import create_react_agent

# ReAct 에이전트는 LangGraph에서 아주 쉽게 만들 수 있습니다.
tools = [calculate_length, reverse_string]
react_agent = create_react_agent(llm, tools)

query = "Calculate the length of 'Agentic' and then reverse the string 'Agentic'."
messages = react_agent.invoke({"messages": [("human", query)]})

print("### ReAct 실행 과정 ###")
for msg in messages['messages']:
    print(f"[{msg.type}]: {msg.content}")

/tmp/ipython-input-1316456668.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  react_agent = create_react_agent(llm, tools)


### ReAct 실행 과정 ###
[human]: Calculate the length of 'Agentic' and then reverse the string 'Agentic'.
[ai]: 
[tool]: 7
[tool]: citnegA
[ai]: The length of 'Agentic' is 7. The reverse of the string 'Agentic' is 'citnegA'.


### 5.2 Reflexion (Self-Reflection)

#### 🐣 비유: "글짓기 후 퇴고하기"
초안을 한 번에 완벽하게 쓰는 사람은 없습니다.
**Reflexion** 패턴은 **작성(Generator)**하고 **비평(Reflector)**하는 과정을 반복합니다.
"내용이 좀 부족한 거 같은데?", "이 부분은 틀렸어"라고 스스로 피드백을 주고 다시 씁니다.

**📚 관련 연구:**
> **Reflexion: an autonomous agent with dynamic memory and self-reflection** (Shinn et al., 2023)
> [[Paper Link]](https://arxiv.org/abs/2303.11366)

In [ ]:
mm("""
graph LR
    User --> Generator
    Generator -->|초안 작성| Reflector
    Reflector -->|비평/피드백| Generator
    Reflector -->|완벽해!| End
""")

In [ ]:
from langchain_core.messages import *
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Annotated
import operator

# 상태 정의: 메시지 목록, 현재 초안, 비평 내용, 반복 횟수
class ReflexionState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    draft: str
    critique: str
    revision_number: int

def generator_node(state):
    # 비평이 있으면 그것을 반영해서 다시 씁니다.
    if state.get("critique"):
        prompt = f"Original Request: {state['messages'][0].content}\nLast Draft: {state['draft']}\nCritique: {state['critique']}\n\nPlease update the draft based on the critique."
    else:
        prompt = f"Write a short essay about: {state['messages'][0].content}"

    response = llm.invoke(prompt)
    return {"draft": response.content, "revision_number": state.get("revision_number", 0) + 1}

def reflector_node(state):
    # 초안을 보고 비평합니다. 너그러운 선생님 역할입니다.
    prompt = f"Identify any missing points or errors in this draft: {state['draft']}. If it is good, say 'PERFECT'."
    response = llm.invoke(prompt)
    return {"critique": response.content}

def should_continue(state):
    # PERFECT를 받거나 3번 이상 수정했으면 끝냅니다.
    if "PERFECT" in state["critique"] or state["revision_number"] > 2:
        return END
    return "Generator"

workflow = StateGraph(ReflexionState)
workflow.add_node("Generator", generator_node)
workflow.add_node("Reflector", reflector_node)

workflow.set_entry_point("Generator")
workflow.add_edge("Generator", "Reflector")
workflow.add_conditional_edges("Reflector", should_continue)

app = workflow.compile()
print("Reflexion Graph 구성 완료!")

Reflexion Graph 구성 완료!


In [ ]:
app.invoke({"messages": [HumanMessage(content="The future of AI")]})

{'messages': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})],
 'draft': 'The simple yet profound greeting "hi!" It\'s a phrase that has been used for centuries to acknowledge the presence of another person, to break the ice in social situations, and to express friendliness and approachability. But have you ever stopped to think about the significance of this seemingly innocuous word?\n\n"Hi!" is more than just a casual greeting; it\'s a way of bridging the gap between individuals, creating a sense of connection and community. When we say "hi!" to someone, we\'re not just acknowledging their physical presence, but also their humanity. We\'re recognizing that they are another person with thoughts, feelings, and experiences, worthy of respect and consideration.\n\nIn many cultures, "hi!" is the first word spoken when meeting someone new, setting the tone for a positive interaction. It\'s a way of showing enthusiasm and interest in getting to know the other person

### 5.3 Plan-and-Execute (계획 후 실행)

#### 🐣 비유: "PM과 개발자"
- **Planner (PM)**: 전체 로드맵을 짭니다. "1단계는 검색, 2단계는 분석, 3단계는 보고서 작성해."
- **Executor (실무자)**: 계획표를 보고 하나씩 처리합니다.

ReAct는 눈앞의 일만 보지만, 이 패턴은 **'큰 그림(Big Picture)'**을 보고 움직이기 때문에 복잡한 프로젝트에 적합합니다.

**📚 관련 연구:**
> **Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models** (Wang et al., 2023)
> [[Paper Link]](https://arxiv.org/abs/2305.04091)

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import AgentState # Recommended import path


class Plan(BaseModel):
    steps: List[str] = Field(description="List of steps to follow")

# 계획을 구조적으로 짜기 위해 Tool Definition 사용
plan_function = {
    "name": "Plan",
    "description": "Generate a step-by-step plan",
    "parameters": Plan.schema()
}

planning_llm = llm.bind_tools([plan_function])

def plan_step(state: AgentState):
    # PM: 계획을 세움
    return {"messages": [AIMessage(content="[Mock Plan] 1. Search for X. 2. Calculate Y.")]}

def execute_step(state: AgentState):
    # Worker: 시킨 대로 실행
    return {"messages": [AIMessage(content="[Mock Execution] Executed Step 1 and 2.")]}

plan_exec_workflow = StateGraph(AgentState)
plan_exec_workflow.add_node("Planner", plan_step)
plan_exec_workflow.add_node("Executor", execute_step)
plan_exec_workflow.set_entry_point("Planner")
plan_exec_workflow.add_edge("Planner", "Executor")
plan_exec_workflow.add_edge("Executor", END)

plan_exec_app = plan_exec_workflow.compile()
print("Plan-and-Execute Graph 구성 완료!")

Plan-and-Execute Graph 구성 완료!


/tmp/ipython-input-648521108.py:12: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  "parameters": Plan.schema()


In [ ]:
plan_exec_app.invoke({"messages": [("human", "What is the capital of France?")]})

{'messages': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='19e7a50c-e754-4a35-910d-ba3e10d1fe46'),
  AIMessage(content='[Mock Plan] 1. Search for X. 2. Calculate Y.', additional_kwargs={}, response_metadata={}, id='6aceadc8-e3b5-4596-943c-a7ff2120dea3', tool_calls=[], invalid_tool_calls=[]),
  AIMessage(content='[Mock Execution] Executed Step 1 and 2.', additional_kwargs={}, response_metadata={}, id='2aa78029-916d-486d-b988-4e984d3cabad', tool_calls=[], invalid_tool_calls=[])]}

### 5.4 Parallelization (병렬 처리)

#### 🐣 비유: "분업화"
한 명이 검색하고, 분석하고, 글 쓰고 하면 하루 종일 걸립니다.
3명이 동시에 각자 맡은 부분을 조사하고, 나중에 팀장이 취합(Aggregate)하면 훨씬 빠릅니다.
이것이 소위 말하는 **Map-Reduce** 구조입니다.

In [ ]:
mm("""
graph TD
    Start --> FanOut
    FanOut --> Agent1
    FanOut --> Agent2
    FanOut --> Agent3
    Agent1 --> Aggregator
    Agent2 --> Aggregator
    Agent3 --> Aggregator
    Aggregator --> End
""")

In [ ]:
from langgraph.graph import StateGraph, START, END

def analyst_1(state):
    return {"messages": [AIMessage(content="[분석가 1] 재무 관점 분석 결과...")]}

def analyst_2(state):
    return {"messages": [AIMessage(content="[분석가 2] 기술 관점 분석 결과...")]}

def aggregator(state):
    # 각 분석가들의 의견을 하나로 모음
    return {"messages": [AIMessage(content="[팀장] 종합 의견: 매수 추천합니다.")]}

parallel_workflow = StateGraph(AgentState)
parallel_workflow.add_node("Analyst1", analyst_1)
parallel_workflow.add_node("Analyst2", analyst_2)
parallel_workflow.add_node("Aggregator", aggregator)

# Fan-out: 시작하자마자 두 분석가가 동시에 출발
parallel_workflow.add_edge(START, "Analyst1")
parallel_workflow.add_edge(START, "Analyst2")

# Fan-in: 두 분석가가 끝나면 팀장에게 보고
parallel_workflow.add_edge("Analyst1", "Aggregator")
parallel_workflow.add_edge("Analyst2", "Aggregator")
parallel_workflow.add_edge("Aggregator", END)

parallel_app = parallel_workflow.compile()
print("Parallel Graph 구성 완료!")

Parallel Graph 구성 완료!


In [ ]:
parallel_app.invoke({"messages": [("human", "재무 분석 결과...")]})

{'messages': [HumanMessage(content='재무 분석 결과...', additional_kwargs={}, response_metadata={}, id='17127da1-0b6b-4f1e-8309-241476e008fb'),
  AIMessage(content='[분석가 1] 재무 관점 분석 결과...', additional_kwargs={}, response_metadata={}, id='b4f4ba5b-44f8-4bae-8387-730bd00e687d', tool_calls=[], invalid_tool_calls=[]),
  AIMessage(content='[분석가 2] 기술 관점 분석 결과...', additional_kwargs={}, response_metadata={}, id='5d1daadf-1eca-4bf0-b08d-5b788c699481', tool_calls=[], invalid_tool_calls=[]),
  AIMessage(content='[팀장] 종합 의견: 매수 추천합니다.', additional_kwargs={}, response_metadata={}, id='84fdb197-f85f-47ea-827c-d0c267f6da59', tool_calls=[], invalid_tool_calls=[])]}

### 5.5 Supervisor Pattern (관리자 패던)

#### 🐣 비유: "팀장님이 작업 배분"
사용자가 요청을 하면, **팀장(Supervisor)**이 누구한테 시킬지 결정합니다.
- "자료 조사가 필요하네? 김대리(Researcher)가 해."
- "이제 코딩이 필요하네? 박과장(Coder)이 해."

Supervisor 자체는 일을 안 하고 교통정리(Routing)만 합니다.

In [ ]:
from typing import Literal

# 1. 슈퍼바이저 에이전트 생성
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  [Researcher, Coder]. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["Researcher", "Coder", "FINISH"]

supervisor_chain = (
    ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{messages}"),
        ("user", f"Given the conversation above, who should act next? Select one of: {options}")
    ])
    | llm
    | StrOutputParser()
)

# 2. 워커 노드 생성 (생략: 위에서 만든 agent 활용)

In [ ]:
supervisor_chain.invoke(input={"messages": "What is the capital of France?"})

'The Researcher should act next.\n\nThey will likely search for and retrieve information about the capital of France from a database or other source. \n\nStatus: The Coder is waiting for input to write code that can verify the answer.'

## 6. Advanced Agentic Features (실전 기능)

데모가 아니라 **실제 서비스**를 만들려면 꼭 필요한 3대장이 있습니다.
1. **Memory**: 대화를 기억해야 합니다.
2. **Human-in-the-loop**: 위험한 일은 허락을 받아야 합니다.
3. **Evaluation**: 내 에이전트가 얼마나 똑똑한지 점수를 매겨야 합니다.

### 6.1 Memory (Persistence via Checkpointers)

#### 🐣 비유: "게임 세이브 포인트"
게임을 하다가 꺼도, **세이브 파일(Checkpoint)**이 있으면 그 자리에서 다시 시작할 수 있죠?
LangGraph도 Checkpointer를 쓰면, 대화의 모든 상태를 DB에 저장합니다.
`thread_id`만 알면 언제든 대화를 이어갈 수 있습니다.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 1. 메모리 저장소 생성 (실제로는 Postgres 같은 DB를 씀)
memory = MemorySaver()

# 2. Agent 만들 때 memory 전달
persistent_agent = create_react_agent(llm, tools, checkpointer=memory)

# 3. thread_id 설정 (이게 사용자 ID나 대화방 ID 역할)
thread_config = {"configurable": {"thread_id": "chulsoo_chat_1"}}

# 4. 대화 테스트
print("--- 첫 번째 대화 ---")
msg1 = persistent_agent.invoke(
    {"messages": [("human", "My name is Chulsoo.")]},
    config=thread_config
)
print(msg1['messages'][-1].content)

print("\n--- 두 번째 대화 (새로 실행해도 기억함) ---")
msg2 = persistent_agent.invoke(
    {"messages": [("human", "What is my name?")]},
    config=thread_config
)
print(msg2['messages'][-1].content)

/tmp/ipython-input-2076484534.py:7: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  persistent_agent = create_react_agent(llm, tools, checkpointer=memory)


--- 첫 번째 대화 ---
The length of the string "My name is Chulsoo" is 13 characters.

--- 두 번째 대화 (새로 실행해도 기억함) ---
The length of the string "What is my name?" is 15 characters.

To answer your original question, your name is Chulsoo.


### 6.2 Human-in-the-loop (Interrupts)

#### 🐣 비유: "부장님 결재 대기"
인턴(AI)이 거래처에 메일을 보내려고 합니다. 그냥 보내면 사고칠 수 있죠?
"부장님(Human), 이거 보낼까요?" 하고 멈춰서 확인받는 기능이 **`interrupt_before`** 입니다.

In [ ]:
def step_1(state):
    print("AI: 파일을 전부 삭제하려고 합니다... (진행 전 멈춤)")
    return {"messages": [AIMessage(content="Delete Plan Ready.")]}

def step_2_action(state):
    print("AI: 쾅! 파일 삭제 완료.")
    return {"messages": [AIMessage(content="Files deleted.")]}

workflow_hitl = StateGraph(AgentState)
workflow_hitl.add_node("planner", step_1)
workflow_hitl.add_node("action", step_2_action)

workflow_hitl.set_entry_point("planner")
workflow_hitl.add_edge("planner", "action")
workflow_hitl.add_edge("action", END)

# 💡 핵심: 'action' 노드 실행 직전에 멈춰라!
app_hitl = workflow_hitl.compile(checkpointer=memory, interrupt_before=["action"])
thread_config_hitl = {"configurable": {"thread_id": "hitl_demo_1"}}

# 실행
print("--- 실행 시작 ---")
app_hitl.invoke({"messages": [("human", "삭제해.")]}, config=thread_config_hitl)

# 지금 멈춰있는 상태입니다. 사용자가 확인 후 승인해야 넘어갑니다.
print("\n--- (사용자 검토 중) 진짜 하시겠습니까? ---")
# app_hitl.invoke(None, config=thread_config_hitl) # 이 줄을 실행해야 넘어감

--- 실행 시작 ---
AI: 파일을 전부 삭제하려고 합니다... (진행 전 멈춤)

--- (사용자 검토 중) 진짜 하시겠습니까? ---


### 6.3 Evaluation with Ragas (성능 평가)

#### 🐣 비유: "채점표"
내가 만든 AI가 멍청한지 똑똑한지 느낌으로만 알면 안되겠죠?
**Ragas**는 시험 출제부터 채점까지 자동으로 해주는 도구입니다.
- **Faithfulness (신뢰성)**: 없는 소리 지어내지 않았나?
- **Answer Relevancy (관련성)**: 동문서답 하지 않았나?
이런 항목들을 수치로 딱 보여줍니다.

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# 0. Ragas 평가용 LLM/Embedding 설정
# Ragas는 자체적으로 API를 호출하려 하므로, 로컬 LLM을 쓰려면 Wrapper로 감싸줘야 합니다.
ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embedding)

# 1. 평가할 질문과 정답 (Ground Truth) 준비
# 이 튜토리얼에서 다룬 내용을 바탕으로 질문을 만듭니다.
eval_questions = [
    "What are the main components of an autonomous agent system?",
    "Explain the concept of 'Planning' in the context of LLM agents."
]

eval_ground_truths = [
    "The main components are Planning, Memory, and Tool use.",
    "Planning usually involves breaking down a large task into smaller, manageable subgoals and reflecting on past actions to improve future results."
]

# 2. RAG 파이프라인을 실제로 돌려서 답변과 검색된 문서를 수집
answers = []
contexts = []

print("### 실제 RAG 파이프라인 평가 데이터 생성 중... ###")

for q in eval_questions:
    print(f"Processing: {q}")

    # 2.1 검색 (Section 3에서 만든 retriever 사용)
    # Ragas 평가를 위해선 검색된 문서의 내용(contexts)이 필요합니다.
    docs = vectorstore.as_retriever().invoke(q)
    context_text = [d.page_content for d in docs]
    contexts.append(context_text)

    # 2.2 답변 생성 (Section 3에서 만든 rag_chain 사용)
    ans = rag_chain.invoke(q)
    answers.append(ans)

# 3. 데이터셋 생성
data_samples = {
    'question': eval_questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': eval_ground_truths
}

dataset = Dataset.from_dict(data_samples)

# 4. 평가 실행 (Evaluate)
print("\n### Ragas 평가 시작 (로컬 LLM 사용) ###")
results = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy],
    llm=ragas_llm,
    embeddings=ragas_embeddings
)

# 5. 결과 확인
print("\n### 평가 결과 ###")
print(results)

# Pandas DataFrame으로 보기 편하게 변환
df_results = results.to_pandas()
df_results.head()

/tmp/ipython-input-2444693692.py:3: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipython-input-2444693692.py:3: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy
/tmp/ipython-input-2444693692.py:9: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(llm)
/tmp/ipython-input-2444693692.py:10: DeprecationWar

### 실제 RAG 파이프라인 평가 데이터 생성 중... ###
Processing: What are the main components of an autonomous agent system?
Processing: Explain the concept of 'Planning' in the context of LLM agents.

### Ragas 평가 시작 (로컬 LLM 사용) ###


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]


### 평가 결과 ###
{'faithfulness': 0.2338, 'answer_relevancy': 0.8931}


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy
0,What are the main components of an autonomous ...,[Overview of a LLM-powered autonomous agent sy...,"According to the text, the main components of ...","The main components are Planning, Memory, and ...",0.181818,1.000000
1,Explain the concept of 'Planning' in the conte...,[Component Three: Tool Use\n\nCase Studies\n\n...,In the context of LLM (Large Language Model) a...,Planning usually involves breaking down a larg...,0.285714,0.786203


## 🎉 마무리
이제 여러분은 단순한 LLM 사용자를 넘어, **'에이전트 설계자(Agent Architect)'**가 되었습니다.
이 노트북에 있는 블록들을 레고처럼 조립해서 자신만의 멋진 AI 서비스를 만들어보세요!